In [1]:
import pandas as pd
import numpy as np
import time, os, sys, gc, json

from Models.framework_utils import *
from Models.diagnostic_utils import *

In [2]:
# load numerapi
public_id, secret_key = get_numerapi_config()
napi, modelnameids = get_napi_and_models(public_id, secret_key)

numerapi ID and Key loaded from config.json


In [5]:
# load data
ds_version = "v4.2"
dataset_loc = os.path.join(os.getcwd(), 'live_datasets', ds_version)
currentRound, isnewRound = get_update_live_data(napi, dataset_loc, ds_version)

np.random.seed(42)
print("# Loading data... ",end='')

# live submission data L* | X = features, P = prediction, I = era indices
live, LI = processData(os.path.join(dataset_loc, 'live_int8.parquet'))

with open(os.path.join(dataset_loc, "features.json"), "r") as f:
    feature_metadata = json.load(f)

ELP = pd.read_parquet(os.path.join(dataset_loc, 'live_example_preds.parquet'), engine="fastparquet")
ids = ELP.index.values
ELP = ELP.values[:,0]

gc.collect()
print("done")

Datasets are up to date.
Current Round: 639
# Loading data... done


In [11]:
import Models

submissions = {}
upload_keys = {}
EMods = []

submissions['example_model'] = ELP

model_modules = [Models.__dict__[x] for x in Models.models]

n_submissions, model_modules = get_currentRound_submissions(
    currentRound, model_modules
)
submissions.update(n_submissions)

In [12]:
submissions

{'example_model': array([0.02616938, 0.05439934, 0.88378323, ..., 0.39666186, 0.29198434,
        0.02987843]),
 'formithlafalcarius': array([0.21976097, 0.27704513, 0.6922522 , ..., 0.6327014 , 0.8043478 ,
        0.15134968]),
 'gravitonglossopteris': array([0.1571193 , 0.30589327, 0.68133116, ..., 0.74170613, 0.72645783,
        0.18452504]),
 'quantaquetzalcoatlus': array([0.14269523, 0.32011127, 0.67288274, ..., 0.7668452 , 0.69348854,
        0.1989491 ]),
 'fraudulinfukuivenatr': array([0.12497424, 0.19276737, 0.23500927, ..., 0.6980218 , 0.97517   ,
        0.24551824]),
 'holmiumhylaeosaur': array([0.39449826, 0.28590563, 0.30362663, ..., 0.8752318 , 0.96713376,
        0.01267257]),
 'iridescentindominus': array([0.02441789, 0.21131259, 0.17401607, ..., 0.5581084 , 0.24675459,
        0.27168763]),
 'lazulilabyrinthodon': array([0.20162787, 0.07036884, 0.30836597, ..., 0.6947249 , 0.91005564,
        0.25973624]),
 'mesozoicmetallurgist': array([0.23253657, 0.06315681, 0.3291

In [13]:
# plot heatmap of corr
corrs = pd.DataFrame(submissions).corr()
# cut the names down to 5 characters
corrs.columns = [x[:5] for x in corrs.columns]
# export to html
corrs.style.background_gradient(cmap='coolwarm',low=0.3,high=0.0,axis=None).format("{:.3f}").to_html('corrs.html')
# print
corrs.style.background_gradient(cmap='coolwarm',low=0.3,high=0.0,axis=None).format("{:.3f}")

,examp,formi,gravi,quant,fraud,holmi,iride,lazul,mesoz,mythr,orich,redst,tanta,titan,diamo,dilit,ether,firer,galli,hedro,krypt,octir,tachy,voidl,wolfr,xylit,bavar,caesi,chrom,galax,iridi,lithi,obsid,pearl,phleb,radiu,shroo,zaniu,zymog,vibra,arcan,energ,resid,sapph,bifro,helix,jovia,kevla,naqua,unobt,wisha,yttri,jadei,vanad,xenon,adama,ferro
example_model,1.000,0.481,0.540,0.545,0.337,0.274,0.279,0.430,0.432,0.307,0.476,0.498,0.391,0.359,0.375,0.426,0.328,0.410,0.242,0.422,0.243,0.325,0.478,0.470,0.431,0.316,0.359,0.382,0.273,0.397,0.240,0.240,0.372,0.239,0.406,0.343,0.323,0.273,0.560,0.571,0.418,0.410,0.352,0.348,0.382,0.319,0.325,0.376,0.471,0.462,0.398,0.409,0.375,0.369,0.317,0.506,0.476
formithlafalcarius,0.481,1.000,0.937,0.893,0.453,0.259,0.249,0.452,0.435,0.263,0.449,0.279,0.472,0.298,0.757,0.683,0.379,0.808,0.301,0.714,0.389,0.453,0.738,0.769,0.332,0.736,0.320,0.472,0.461,0.491,0.278,0.305,0.482,0.304,0.479,0.498,0.484,0.309,0.482,0.662,0.606,0.633,0.668,0.583,0.583,0.567,0.641,0.608,0.617,0.644,0.680,0.599,0.360,0.371,0.350,0.611,0.574
gravitonglossopteris,0.540,0.937,1.000,0.994,0.386,0.298,0.310,0.486,0.487,0.317,0.500,0.356,0.446,0.360,0.709,0.749,0.433,0.755,0.376,0.753,0.423,0.493,0.804,0.806,0.407,0.630,0.387,0.533,0.392,0.531,0.318,0.346,0.521,0.351,0.541,0.471,0.457,0.359,0.519,0.764,0.684,0.685,0.634,0.648,0.656,0.629,0.608,0.656,0.698,0.699,0.645,0.665,0.352,0.360,0.334,0.691,0.677
quantaquetzalcoatlus,0.545,0.893,0.994,1.000,0.356,0.304,0.322,0.485,0.492,0.326,0.504,0.373,0.427,0.371,0.676,0.751,0.439,0.719,0.391,0.746,0.423,0.494,0.804,0.797,0.422,0.581,0.400,0.539,0.360,0.531,0.324,0.351,0.521,0.357,0.547,0.451,0.437,0.366,0.518,0.778,0.692,0.684,0.607,0.653,0.662,0.633,0.582,0.655,0.706,0.698,0.618,0.669,0.341,0.347,0.321,0.699,0.693
fraudulinfukuivenatr,0.337,0.453,0.386,0.356,1.000,0.388,0.434,0.826,0.761,0.446,0.748,0.170,0.968,0.384,0.387,0.309,0.184,0.400,0.139,0.335,0.208,0.197,0.324,0.349,0.160,0.399,0.158,0.292,0.356,0.317,0.165,0.203,0.311,0.198,0.296,0.358,0.348,0.181,0.342,0.329,0.242,0.263,0.308,0.226,0.232,0.218,0.296,0.252,0.259,0.281,0.330,0.249,0.304,0.294,0.256,0.476,0.302
holmiumhylaeosaur,0.274,0.259,0.298,0.304,0.388,1.000,0.277,0.535,0.543,0.240,0.589,0.225,0.468,0.259,0.205,0.236,0.167,0.217,0.240,0.233,0.212,0.215,0.246,0.243,0.258,0.171,0.149,0.248,0.168,0.239,0.178,0.230,0.240,0.196,0.244,0.208,0.202,0.200,0.247,0.323,0.240,0.231,0.185,0.261,0.216,0.229,0.170,0.208,0.265,0.255,0.202,0.280,0.173,0.156,0.121,0.405,0.337
iridescentindominus,0.279,0.249,0.310,0.322,0.434,0.277,1.000,0.640,0.655,0.270,0.542,0.190,0.542,0.276,0.225,0.303,0.213,0.214,0.307,0.289,0.346,0.263,0.278,0.266,0.230,0.166,0.096,0.294,0.147,0.268,0.243,0.269,0.275,0.242,0.284,0.205,0.205,0.207,0.255,0.345,0.303,0.285,0.209,0.313,0.287,0.293,0.199,0.270,0.320,0.302,0.222,0.322,0.178,0.161,0.129,0.487,0.399
lazulilabyrinthodon,0.430,0.452,0.486,0.485,0.826,0.535,0.640,1.000,0.994,0.626,0.966,0.301,0.939,0.530,0.389,0.423,0.267,0.398,0.265,0.422,0.327,0.315,0.432,0.431,0.263,0.338,0.246,0.408,0.303,0.413,0.257,0.317,0.399,0.288,0.420,0.365,0.352,0.256,0.427,0.473,0.361,0.356,0.317,0.346,0.344,0.326,0.303,0.340,0.386,0.382,0.339,0.370,0.313,0.292,0.252,0.638,0.463
mesozoicmetallurgist,0.432,0.435,0.487,0.492,0.761,0.543,0.655,0.994,1.000,0.638,0.970,0.316,0.896,0.539,0.374,0.429,0.272,0.382,0.279,0.423,0.337,0.326,0.436,0.430,0.272,0.314,0.254,0.414,0.281,0.415,0.265,0.328,0.401,0.295,0.428,0.352,0.339,0.260,0.427,0.483,0.371,0.361,0.306,0.357,0.353,0.335,0.292,0.344,0.396,0.387,0.328,0.380,0.303,0.280,0.241,0.645,0.477
mythrilmorelladon,0.307,0.263,0.317,0.326,0.446,0.240,0.270,0.626,0.638,1.000,0.588,0.219,0.543,0.288,0.243,0.312,0.242,0.241,0.269,0.300,0.289,0.269,0.310,0.298,0.216,0.186,0.204,0.274,0.143,0.271,0.173,0.250,0.257,0.211,0.288,0.207,0.195,0.207,0.298,0.353,0.236,0.227,0.181,0.246,0.229,0.233,0.173,0.220,0.254,0.244,0.195,0.259,0.159,0.154,0.129,0.455,0.374
